In [4]:
%matplotlib inline
import geopandas
import pandas
import os
import glob

import dateutil
import datetime

from simpledbf import Dbf5
import pandas_access as mdb


pandas.set_option("display.max_columns",999)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import parcels_import
parcels = parcels_import.parcels
parcels.head()

## Read in CSB data
* subset data to only last 180 days

In [ ]:
csb = mdb.read_table("data/open-data-extracted/csb/cityworks.mdb", "dbo_vw_public")

start_search_timestamp = (datetime.datetime.now() - datetime.timedelta(days=180))
csb.loc[:,'StartDate'] = csb.DATETIMEINIT.map(dateutil.parser.parse)

csb = csb[csb.StartDate.apply(lambda x: x.to_pydatetime().timestamp() > start_search_timestamp.timestamp())]
csb.reset_index(inplace=True,drop=True)

csb.tail()

### Join CSB and Parcel data
* subset to Marine Villa only

In [ ]:
problem_address = csb.groupby('PROBADDRESS').REQUESTID.count().sort_values(ascending=False)

problem_address = pandas.DataFrame(problem_address)
problem_address.reset_index(level=0, inplace=True)

problem_address.loc[:,'PROBADDRESS'] = problem_address.PROBADDRESS.apply(lambda x: x.replace('.',''))
problem_address.rename(columns=({'REQUESTID':'CSB_Isues_Last180Days'}),inplace=True)

parcels = pandas.merge(parcels,problem_address,left_on='SITEADDR',right_on='PROBADDRESS',how='left')

In [ ]:
parcels_MV = parcels[(~parcels.PROBADDRESS.isnull()) & (parcels.NHD_NAME.isin(['Marine Villa']))][['HANDLE','geometry','UPDATED','OwnerName','OwnerAddr','OwnerCity','OwnerState','SITEADDR','LowAddrNum','StName','NHD_NAME','PROBADDRESS']]
parcels_MV = pandas.merge(parcels_MV,csb,on='PROBADDRESS')

In [ ]:
parcels_MV.reset_index(inplace=True,drop=True)

In [ ]:
parcels_MV.to_csv('parcels_MV_v1.csv',index=False)

In [ ]:
parcels_MV.groupby('CALLERTYPE').PROBADDRESS.count().sort_values(ascending=False)

In [ ]:
parcels_MV.groupby('PROBLEMCODE').PROBADDRESS.count().sort_values(ascending=False)

In [ ]:
parcels[parcels.apply(lambda x: 'Marine Villa' in x['NHD_NAME'],axis=1)]

In [ ]:
parcels.groupby('NHD_NAME').CSB_Isues_Last180Days.sum().sort_values(ascending=False).head(30)

In [ ]:
parcels_per_neighborhood = parcels.groupby('NHD_NAME').HANDLE.count().sort_values(ascending=False)
print(sum(parcels_per_neighborhood))
parcels_per_neighborhood

In [ ]:
(parcels.groupby('NHD_NAME').CSB_Isues_Last180Days.sum() / parcels_per_neighborhood).sort_values(ascending=False).head(30)

In [ ]:
(parcels.groupby('NHD_NAME').CSB_Isues_Last180Days.sum() / parcels_per_neighborhood).sort_values(ascending=False).head(30).plot('bar',figsize=(30,15),title='CSB Requests Rate for last 180 Days by Parcel Count Per Neighborhood')

In [ ]:
parcels.groupby('NHD_NAME').CSB_Isues_Last180Days.sum().sort_values(ascending=False).head(30).astype(int)

In [ ]:
parcels[parcels.NHD_NAME=='Marine Villa'].REQUESTID.sum()

In [ ]:
parcels[parcels.NHD_NAME=='Marine Villa'].groupby('OWNERNAME').HANDLE.count().sort_values(ascending=False)

In [ ]:
parcels.columns.Address.unique()

In [ ]:
parcels.to_csv('parcels_out.csv')

In [ ]:
parcels = pandas.merge(parcels,tax_sales,left_on='SITEADDR',right_on='Address',how='left')

In [ ]:
parcels.loc[:,'geometry_centroid'] = parcels.centroid